In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium

### Data оруулах

In [21]:
df = pd.read_csv('input/ads_data_2025-06-19_allsheet.csv')
df[df['mytitle'].str.contains('express tower', case=False, na=False)]

,№,ad_id,mytitle,district,khoroo,mylocation,price,area,price_m2,progress_cons,clean_coords,url
368,369,9429388,Express tower,Сүхбаатар,1.0,Талбай,599.0,80,7.49,Ашиглалтад орсон,47.929336 106.913516,https://www.unegui.mn/adv/9408526_dolgoon-nuur...
374,375,9429008,Express tower,Сүхбаатар,1.0,Талбай,600.0,80,7.50,Ашиглалтад орсон,47.912432 106.921754,https://www.unegui.mn/adv/9422716_sbd1-r-khoro...
382,383,9195830,Express tower,Сүхбаатар,1.0,Талбай,1500.0,230,6.52,Ашиглалтад орсон,47.912432 106.921754,https://www.unegui.mn/adv/9117819_olympic-resi...
390,391,9195207,Express tower,Сүхбаатар,1.0,Талбай,1500.0,230,6.52,Ашиглалтад орсон,47.9188290307491 106.917622814522,https://www.unegui.mn/adv/8999119_minister-tow...
393,394,9203763,Express tower,Сүхбаатар,1.0,Талбай,1500.0,230,6.52,Ашиглалтад орсон,47.9188290307491 106.917622814522,https://www.unegui.mn/adv/8797532_khotyn-tovd-...
394,395,9197082,Express tower,Сүхбаатар,1.0,Талбай,1500.0,230,6.52,Ашиглалтад орсон,47.9188290307491 106.917622814522,https://www.unegui.mn/adv/9441646_ministertowe...
401,402,9197295,Express tower,Сүхбаатар,1.0,Талбай,1500.0,230,6.52,Ашиглалтад орсон,47.922769 106.917593,https://www.unegui.mn/adv/9132959_metro-moll-n...
407,408,9411002,Express tower,Сүхбаатар,1.0,Талбай,7.5,80,7.50,Ашиглалтад орсон,47.925535 106.917694,https://www.unegui.mn/adv/9079040_khotyn-tovd-...
412,413,9411698,Express tower,Сүхбаатар,1.0,Талбай,600.0,80,7.50,Ашиглалтад орсон,47.9264199716145 106.915377110215,https://www.unegui.mn/adv/7411288_metromollyn-...
413,414,9406003,Express tower,Сүхбаатар,1.0,Талбай,520.0,80,6.50,Ашиглалтад орсон,47.925535 106.917694,https://www.unegui.mn/adv/9168546_metro-mall-k...


### Оффисын нэрээр дундаж үнэ болон зарын тоог харуулах

In [4]:
# mytitle дээр бүлэглээд дундаж үнэ ба зарын тоог хамт гаргах
summary = df.groupby('mytitle').agg(
    avg_price_m2=('price_m2', 'mean'),
    ad_count=('price_m2', 'count')
).sort_values('avg_price_m2', ascending=False)

# Хамгийн өндөр үнэтэй 10 оффисыг харах
summary.tail(10)

,avg_price_m2,ad_count
mytitle,,
Titan Plus,4.551429,7
Sydney tower,4.500000,2
Хос даль,4.500000,2
Метро бизнес центр,4.470000,1
Москва төв,4.263333,3
Titan Center,4.240000,2
Хоймор оффис,3.985000,4
Dragon center,3.980000,1
Union Building,3.945714,14


# Оффис үнэлгээний аргачлал

In [5]:
df1 = pd.read_excel('D:/Projects/Office/Office аргачлал.xlsx', sheet_name=4)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   №                190 non-null    float64
 1   district         193 non-null    object 
 2   khoroo           193 non-null    int64  
 3   number           187 non-null    object 
 4   zip code         165 non-null    float64
 5   Хаяг нэгтгэсэн   0 non-null      float64
 6   office_name      184 non-null    object 
 7   Бүтээц           193 non-null    object 
 8   floor            192 non-null    float64
 9   op_date          183 non-null    object 
 10  Гудамжны нэр     187 non-null    object 
 11  Координат        182 non-null    object 
 12  Үнэлгээ          2 non-null      float64
 13  Үнэлгээ
2024.12  2 non-null      float64
 14  Large            2 non-null      float64
 15  Байрны дугаар    0 non-null      float64
dtypes: float64(8), int64(1), object(7)
memory usage: 24.3+ KB


In [6]:
# 1. df-ээс mytitle бүрийн дундаж үнэ гаргаж dict болгон хувиргана
avg_price_dict = df.groupby('mytitle')['price_m2'].mean().to_dict()

# 2. df1['office_name']-ийг ашиглан дундаж үнийг 'Үнэлгээ' баганад оруулах
df1['Үнэлгээ'] = df1['office_name'].map(avg_price_dict)
df1.to_excel('average_price.xlsx', index=False)

In [7]:
# mytitle дээр бүлэглээд дундаж үнэ ба зарын тоог хамт гаргах
summary = df.groupby('mytitle').agg(
    avg_price_m2=('price_m2', 'min'),
    ad_count=('price_m2', 'count')
).sort_values('avg_price_m2', ascending=False)

# Хамгийн өндөр үнэтэй 10 оффисыг харах
summary.head(10)

,avg_price_m2,ad_count
mytitle,,
Minister Tower,13.47,4
Park place,13.00,1
Ayud tower,12.00,1
MERU tower,10.65,1
Paragon,10.00,1
Galaxy tower,10.00,2
Gem castle,9.80,9
Soyombo tower,9.60,17
Sumo tower,9.50,2


In [10]:
### Үнэлж буй хөрөнгүүдийг газрын зурагт харуулах

In [8]:
import folium

# Газрын зургийн төв координат
map_center = [47.918873, 106.917701]
m = folium.Map(location=map_center, zoom_start=13)

# Marker нэмэх
for idx, row in df1.iterrows():
    coord = row['Координат']
    if isinstance(coord, str) and ',' in coord:
        try:
            lat, lon = map(float, coord.split(','))
            
            # Popup мэдээлэл
            popup_text = f"""
            <b>Office name:</b> {row.get('office_name', '')}<br>
            <b>Дүүрэг:</b> {row.get('district', '')}<br>
            <b>Хороо:</b> {row.get('khoroo', '')}<br>
            <b>Байрны дугаар:</b> {row.get('number', '')}<br>
            <b>Ашиглалтад орсон:</b> {row.get('op_date', '')}<br>            
            <b>Давхар:</b> {row.get('floor', '')}<br>
            <b>Үнэлгээ:</b> {row.get('Үнэлгээ', '')}
            """
            
            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(popup_text, max_width=300),
                icon=folium.Icon(color='blue', icon='building')
            ).add_to(m)
        except ValueError:
            print(f"Invalid coordinate: {coord}")

# Газрын зургийг HTML файлд хадгалах
m.save("office_map_with_details.html")

,№,ad_id,mytitle,district,khoroo,mylocation,price,area,price_m2,progress_cons,clean_coords,url
0,1,9299916,Tasgan tower,Чингэлтэй,NaN,Баянбүрд,482.04,74,6.51,Ашиглалтад орсон,47.9278806292485 106.90571821241,https://www.unegui.mn/adv/9299916_tasgan-tover...
1,2,9451753,Tasgan tower,Чингэлтэй,5.0,5-р сургууль,254.41,39,6.52,Ашиглалтад орсон,47.92397 106.902687,https://www.unegui.mn/adv/9451753_tasgan-taurt...
2,3,9457746,Uniq center,Хан-Уул,17.0,Зайсан хойно,7.30,65,7.30,Ашиглалтад орсон,47.8927222013864 106.923521825545,https://www.unegui.mn/adv/9457746_khan-uul-i-m...
3,4,9344603,NaN,Сүхбаатар,1.0,Талбай,3200.00,320,10.00,Ашиглалтад орсон,47.9188290307491 106.917622814522,https://www.unegui.mn/adv/9455848_talbain-baru...
4,5,9432720,Tasgan tower,Чингэлтэй,5.0,5-р сургууль,175.00,34,5.15,Ашиглалтад орсон,47.92397 106.902687,https://www.unegui.mn/adv/9432720_tasgan-taurt...
...,...,...,...,...,...,...,...,...,...,...,...,...
1066,1067,9312911,Paradise plaza,Баянзүрх,NaN,Баянмонгол,1260.00,200,6.30,Ашиглалтад орсон,47.904759 106.930979,https://www.unegui.mn/adv/9312911_baianmongoly...
1067,1068,9057555,Misheel city,Хан-Уул,20.0,Зайсан хойно,7.50,107,7.50,Ашиглалтад орсон,47.8919232454507 106.887139941548,https://www.unegui.mn/adv/9057555_misheel-siti...
1068,1069,9364327,NaN,Баянгол,NaN,Нарны хороолол,290.00,56,5.18,Ашиглалтад орсон,47.90771 106.899413,https://www.unegui.mn/adv/9364327_narny-khoroo...
1069,1070,9217759,Business tower,Сүхбаатар,NaN,220 мянгат,1100.00,120,9.17,Ашиглалтад орсон,47.912432 106.921754,https://www.unegui.mn/adv/9394639_ayud-tower-t...
